In [1]:
import numpy as np
import pandas as pd
from mne.stats import fdr_correction
import statsmodels.stats.multitest as multi
from scipy.stats import mannwhitneyu
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("RNA.Result.csv",sep=';', decimal=',')
df

,PatientID,Tumor,A1BG,A2LD1,A2M,A4GALT,AAAS,AACS,AADAC,AADAT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22
0,TCGA-05-4244-01A-01R-1107-07,1,26.0302,135.5022,9844.7858,130.9015,651.0167,803.5605,3.4907,53.0587,...,221.1642,517.6717,39.794000000000004,292.5212,959.2460000000001,1088.0531,2837.9440000000004,871.2802,575.2683,6.6323
1,TCGA-05-4249-01A-01R-1107-07,1,120.1349,89.0629,25712.6617,578.5562,604.3128,999.9976,266.9021,185.4471,...,198.5893,287.5074,62.1376,493.5597,1179.3275,787.5061,2351.25,1138.117,690.2752,179.9738
2,TCGA-05-4250-01A-01R-1107-07,1,50.8597,151.1332,16943.6359,356.4619,588.5766,754.0413,26.1958,81.9033,...,520.8986,1046.8374,60.0182,365.4149,843.9028,475.17199999999997,5437.4534,1170.5214,532.8691,6.3003
3,TCGA-05-4382-01A-01R-1206-07,1,145.9037,112.0685,23326.2727,554.4677,608.3112,1079.9796,3.5896,46.6643,...,429.6299,753.0902,40.4424,346.7517,946.6872,908.1593,6770.1537,1169.2401,663.8297,35.1777
4,TCGA-05-4384-01A-01R-1755-07,1,127.3671,87.5748,48314.5571,152.7184,665.8522,709.8351,34.8198,100.7941,...,258.3018,153.9401,80.0244,563.2254,1320.7086,778.8638,3341.4783,1737.3244,723.2743,378.1307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,TCGA-91-6831-11A-02R-1858-07,0,62.9634,47.3918,79556.0155,338.5128,374.0289,406.2153,196.4484,88.7902,...,134.2952,54.38399999999999,45.505,405.1054,1053.2741,1169.8113,4502.7747,1066.5927,553.8291,84.3507
570,TCGA-91-6835-11A-01R-1858-07,0,76.7722,55.9789,77953.782,561.7934,604.3819,613.7765,125.8868,108.3503,...,177.7634,85.1771,69.5196,367.6396,1246.3419999999999,894.36,4142.3648,1833.8138,545.5095,144.6759
571,TCGA-91-6836-11A-01R-1858-07,0,79.8192,61.3333,55831.1499,296.1186,560.2033,718.0291,117.1973,135.9489,...,117.9162,61.7239,56.2547,346.1228,1398.555,1154.7845,4739.4605,1862.6564,653.1799,85.9447
572,TCGA-91-6847-11A-01R-1949-07,0,67.6923,61.4389,39359.095,267.8733,540.2715,610.8597,470.5882,105.8824,...,150.2262,64.2534,52.4887,360.181,1340.2715,1035.2941,4609.0498,1458.8235,558.371,62.4434


In [3]:
#df2
df1 = df[df['Tumor'] == 1]
df2 = df[df['Tumor'] == 0]

In [4]:
PValue={}
Stat={}
alpha=0.05
df3=[]

In [5]:
for col in df:
    if col!="PatientID" and col !="Tumor":
        Stat[col],PValue[col] = mannwhitneyu(df1[col],df2[col])
        if PValue[col]<=alpha:
            df3.append({'Name':col,'PValue':PValue[col]})

In [6]:
df3=pd.DataFrame(df3)

In [7]:
df3

,Name,PValue
0,A1BG,3.223620e-03
1,A2LD1,1.239438e-05
2,A2M,6.875438e-04
3,A4GALT,1.595041e-02
4,AAAS,8.386944e-08
...,...,...
10456,ZXDB,4.192365e-02
10457,ZXDC,1.901728e-03
10458,ZYG11B,4.020253e-03
10459,ZYX,1.321236e-12


In [8]:
pval=df3['PValue']
pval=list(pval)

In [9]:
PValue_holm = multi.multipletests(pval,alpha=alpha,method='holm')
Reject_holm = PValue_holm[0]
PValue_holm = PValue_holm[1]

In [10]:
for index,row in df3.iterrows():
    df3.at[index, 'reject_HOLM'] = Reject_holm.item(index)
    df3.at[index, 'PValue_HOLM'] = PValue_holm.item(index)

In [11]:
df3.reject_HOLM.value_counts()

False    6306
True     4155
Name: reject_HOLM, dtype: int64

In [12]:
PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_bh')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]

In [13]:
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_bh'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_bh'] = PValue_fdr.item(index)

In [14]:
'''PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_by')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_by'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_by'] = PValue_fdr.item(index)'''

"PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_by')\nReject_fdr = PValue_fdr[0]\nPValue_fdr = PValue_fdr[1]\nfor index,row in df3.iterrows():\n df3.at[index, 'reject_fdr_by'] = Reject_fdr.item(index)\n df3.at[index, 'PValue_fdr_by'] = PValue_fdr.item(index)"

In [15]:
'''PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_tsbh')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_tsbh'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_tsbh'] = PValue_fdr.item(index)
'''

"PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_tsbh')\nReject_fdr = PValue_fdr[0]\nPValue_fdr = PValue_fdr[1]\nfor index,row in df3.iterrows():\n df3.at[index, 'reject_fdr_tsbh'] = Reject_fdr.item(index)\n df3.at[index, 'PValue_fdr_tsbh'] = PValue_fdr.item(index)\n"

In [16]:
'''PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_tsbky')
Reject_fdr = PValue_fdr[0]
PValue_fdr = PValue_fdr[1]
for index,row in df3.iterrows():
 df3.at[index, 'reject_fdr_tskby'] = Reject_fdr.item(index)
 df3.at[index, 'PValue_fdr_tskby'] = PValue_fdr.item(index)'''


"PValue_fdr = multi.multipletests(pval,alpha=alpha,method='fdr_tsbky')\nReject_fdr = PValue_fdr[0]\nPValue_fdr = PValue_fdr[1]\nfor index,row in df3.iterrows():\n df3.at[index, 'reject_fdr_tskby'] = Reject_fdr.item(index)\n df3.at[index, 'PValue_fdr_tskby'] = PValue_fdr.item(index)"

In [17]:
df3

,Name,PValue,reject_HOLM,PValue_HOLM,reject_fdr_bh,PValue_fdr_bh
0,A1BG,3.223620e-03,False,1.000000e+00,True,4.435976e-03
1,A2LD1,1.239438e-05,False,7.586599e-02,True,2.986127e-05
2,A2M,6.875438e-04,False,1.000000e+00,True,1.108398e-03
3,A4GALT,1.595041e-02,False,1.000000e+00,True,1.840066e-02
4,AAAS,8.386944e-08,True,6.444527e-04,True,3.158237e-07
...,...,...,...,...,...,...
10456,ZXDB,4.192365e-02,False,1.000000e+00,True,4.285774e-02
10457,ZXDC,1.901728e-03,False,1.000000e+00,True,2.758838e-03
10458,ZYG11B,4.020253e-03,False,1.000000e+00,True,5.397314e-03
10459,ZYX,1.321236e-12,True,1.224389e-08,True,1.156607e-11


In [18]:
#df3.to_csv(r'C:\Users\Macie\Praca_Dyplomowa\PValue.RNA.csv',sep=';', index=False)
#df3 = pd.read_csv("PValue.RNA.csv",sep=';', decimal=',')

In [19]:
#usuniecie wartosci PValue < 0.05
dfR = df3.loc[(df3['reject_HOLM'] == True)&(df3['reject_fdr_bh'] == True)]
dfR

,Name,PValue,reject_HOLM,PValue_HOLM,reject_fdr_bh,PValue_fdr_bh
4,AAAS,8.386944e-08,True,6.444527e-04,True,3.158237e-07
9,AAMP,1.879714e-17,True,1.879902e-13,True,4.265443e-16
10,AARS2,2.409810e-14,True,2.313659e-10,True,2.927878e-13
11,AARSD1,1.791644e-22,True,1.846647e-18,True,1.209186e-20
12,AARS,3.613669e-14,True,3.459724e-10,True,4.252261e-13
...,...,...,...,...,...,...
10450,ZSWIM5,9.504043e-07,True,6.726962e-03,True,2.937996e-06
10452,ZSWIM7,1.179753e-08,True,9.532406e-05,True,5.181108e-08
10453,ZW10,1.297158e-14,True,1.252014e-10,True,1.669074e-13
10454,ZWILCH,2.121352e-25,True,2.203024e-21,True,2.882009e-23


In [20]:
#Stworzenie rankingu najwazniejszych genów (holm)
dfR_holm = dfR.sort_values(by ='PValue_HOLM' )
dfR_holm

,Name,PValue,reject_HOLM,PValue_HOLM,reject_fdr_bh,PValue_fdr_bh
2214,CSTF2,1.279451e-32,True,1.338433e-28,True,7.459975e-29
2784,EIF2AK1,1.426245e-32,True,1.491852e-28,True,7.459975e-29
3692,GORASP2,5.372632e-32,True,5.619236e-28,True,1.873437e-28
4331,IRAK1,3.148954e-31,True,3.293176e-27,True,8.235303e-28
970,C13orf1,4.675400e-31,True,4.889065e-27,True,8.390369e-28
...,...,...,...,...,...,...
2480,DHRS13,7.849274e-06,True,4.954462e-02,True,1.978108e-05
259,ALAS1,7.878794e-06,True,4.971519e-02,True,1.984592e-05
3651,GNB1L,7.878797e-06,True,4.971519e-02,True,1.984592e-05
3792,GRIK4,7.887747e-06,True,4.975591e-02,True,1.986368e-05


In [21]:
#wziecie 100 najwazniejszych genów (holm)
dfR_holm = dfR_holm.head(100)
dfR_holm

,Name,PValue,reject_HOLM,PValue_HOLM,reject_fdr_bh,PValue_fdr_bh
2214,CSTF2,1.279451e-32,True,1.338433e-28,True,7.459975e-29
2784,EIF2AK1,1.426245e-32,True,1.491852e-28,True,7.459975e-29
3692,GORASP2,5.372632e-32,True,5.619236e-28,True,1.873437e-28
4331,IRAK1,3.148954e-31,True,3.293176e-27,True,8.235303e-28
970,C13orf1,4.675400e-31,True,4.889065e-27,True,8.390369e-28
...,...,...,...,...,...,...
8631,STARD7,1.854191e-24,True,1.922054e-20,True,2.020489e-22
426,APH1B,1.901978e-24,True,1.971400e-20,True,2.051195e-22
4109,HSD17B4,1.950985e-24,True,2.022001e-20,True,2.082577e-22
6061,NPLOC4,2.105635e-24,True,2.182069e-20,True,2.224954e-22


In [22]:
#Stworzenie rankingu i wziecie 100 najwazniejszych genów (fdr)
dfR_fdr = dfR.sort_values(by ='PValue_fdr_bh' )
dfR_fdr = dfR_fdr.head(100)
dfR_fdr

,Name,PValue,reject_HOLM,PValue_HOLM,reject_fdr_bh,PValue_fdr_bh
2784,EIF2AK1,1.426245e-32,True,1.491852e-28,True,7.459975e-29
2214,CSTF2,1.279451e-32,True,1.338433e-28,True,7.459975e-29
3692,GORASP2,5.372632e-32,True,5.619236e-28,True,1.873437e-28
4331,IRAK1,3.148954e-31,True,3.293176e-27,True,8.235303e-28
8433,SNX1,4.812371e-31,True,5.031815e-27,True,8.390369e-28
...,...,...,...,...,...,...
5995,NKRF,1.838528e-24,True,1.906002e-20,True,2.020489e-22
426,APH1B,1.901978e-24,True,1.971400e-20,True,2.051195e-22
4109,HSD17B4,1.950985e-24,True,2.022001e-20,True,2.082577e-22
6061,NPLOC4,2.105635e-24,True,2.182069e-20,True,2.224954e-22


In [23]:
dfR_holm.to_csv(r'C:\Users\Macie\Praca_Dyplomowa\dfR_holm.csv',sep=';', index=False)
dfR_fdr.to_csv(r'C:\Users\Macie\Praca_Dyplomowa\dfR_fdr.csv',sep=';', index=False)

In [24]:
#wziecie najwazniejszych genów
ImportantValues = dfR_fdr['Name'].tolist()
dfIV = df.loc[:,ImportantValues]
dfIV

,EIF2AK1,CSTF2,GORASP2,IRAK1,SNX1,C13orf1,KIF1C,GARS,SLC25A39,ADD1,...,ARL13B,ALG6,TRIM35,PTCD1,STARD7,NKRF,APH1B,HSD17B4,NPLOC4,HSP90AB1
0,6818.1098,772.1442,2348.547,4379.7888,1655.9909,234.2264,1515.3155,2435.8146,3600.3142,4007.3305,...,233.8773,371.4111,268.4353,365.3443,4562.4575,559.5602,121.8256,2452.221,4543.852,30568.8105
1,3346.7627,494.8475,5591.7444,1324.8519,2624.5912,200.2571,2713.1293,1569.861,2354.1476,4386.0165,...,202.5108,222.7941,471.6666,308.6535,3337.9185,379.9089,266.5802,1916.2864,3059.8767,33322.521
2,5447.9085,677.4434,3943.2977,3398.1597,1827.0745,318.3288,3529.1387,3322.8882,4692.6967,5796.568,...,270.9111,161.8171,402.2217,438.2293,3135.6147,443.6707,347.5089,1464.9755,3358.3686,34958.7996
3,6730.5776,421.6539,3877.4452,5879.9423,2234.3829,285.9685,4327.5763,2248.5018,2881.7006,5654.5178,...,391.7409,274.0033,346.5124,455.6136,2852.5892,160.5731,421.4146,1980.2419,4312.9787,21074.0808
4,5373.090999999999,364.0806,3034.2089,2764.8137,3251.6799,362.24800000000005,4911.4233,1966.402,2376.2981,4850.9469,...,166.7685,209.5296,472.8161,403.934,3010.5559,222.9688,448.3812,3263.8974,4005.4979,18775.8094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2415.8713,188.6792,1479.4673,1002.2198,4497.2253,751.3873,6762.4861,1358.4906,790.2331,6682.5749,...,1032.1865,168.7014,398.4462,190.4772,5005.7381,243.0633,589.3452,4625.9711,1654.828,18588.2353
570,3258.6457,249.2684,1822.5403,1630.2654,3695.8111,379.5393,4647.791,1252.605,1689.7642,7309.5766,...,479.1214,154.0704,462.8376,278.9176,4172.2457,226.0952,616.2817,2688.0904,2113.1447,22259.4176
571,2776.9988,180.4839,1733.7394,1388.3979,4187.0704,613.3328,6153.6418,1283.7016,1502.4699,6661.496999999999,...,448.4752,192.2036,388.3139,247.3333,4787.2458,245.3331,737.5619,4787.9021,2144.7114,17821.8093
572,2952.1629,200.0,1902.2624,1334.8416,4635.2941,925.7919,9858.8235,1237.1041,1302.2624,7250.6787,...,593.6652,143.8914,565.6109,303.0769,7069.6742,229.8643,607.2398,4186.4253,2135.7466,21454.2986


In [25]:
#for col in dfIV:
#    print(col)

In [26]:
dfTEMP = df.iloc[:,0:2]

In [27]:
df = pd.concat([dfTEMP, dfIV], axis=1)

In [31]:
dfIV

,EIF2AK1,CSTF2,GORASP2,IRAK1,SNX1,C13orf1,KIF1C,GARS,SLC25A39,ADD1,...,ARL13B,ALG6,TRIM35,PTCD1,STARD7,NKRF,APH1B,HSD17B4,NPLOC4,HSP90AB1
0,6818.1098,772.1442,2348.547,4379.7888,1655.9909,234.2264,1515.3155,2435.8146,3600.3142,4007.3305,...,233.8773,371.4111,268.4353,365.3443,4562.4575,559.5602,121.8256,2452.221,4543.852,30568.8105
1,3346.7627,494.8475,5591.7444,1324.8519,2624.5912,200.2571,2713.1293,1569.861,2354.1476,4386.0165,...,202.5108,222.7941,471.6666,308.6535,3337.9185,379.9089,266.5802,1916.2864,3059.8767,33322.521
2,5447.9085,677.4434,3943.2977,3398.1597,1827.0745,318.3288,3529.1387,3322.8882,4692.6967,5796.568,...,270.9111,161.8171,402.2217,438.2293,3135.6147,443.6707,347.5089,1464.9755,3358.3686,34958.7996
3,6730.5776,421.6539,3877.4452,5879.9423,2234.3829,285.9685,4327.5763,2248.5018,2881.7006,5654.5178,...,391.7409,274.0033,346.5124,455.6136,2852.5892,160.5731,421.4146,1980.2419,4312.9787,21074.0808
4,5373.090999999999,364.0806,3034.2089,2764.8137,3251.6799,362.24800000000005,4911.4233,1966.402,2376.2981,4850.9469,...,166.7685,209.5296,472.8161,403.934,3010.5559,222.9688,448.3812,3263.8974,4005.4979,18775.8094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2415.8713,188.6792,1479.4673,1002.2198,4497.2253,751.3873,6762.4861,1358.4906,790.2331,6682.5749,...,1032.1865,168.7014,398.4462,190.4772,5005.7381,243.0633,589.3452,4625.9711,1654.828,18588.2353
570,3258.6457,249.2684,1822.5403,1630.2654,3695.8111,379.5393,4647.791,1252.605,1689.7642,7309.5766,...,479.1214,154.0704,462.8376,278.9176,4172.2457,226.0952,616.2817,2688.0904,2113.1447,22259.4176
571,2776.9988,180.4839,1733.7394,1388.3979,4187.0704,613.3328,6153.6418,1283.7016,1502.4699,6661.496999999999,...,448.4752,192.2036,388.3139,247.3333,4787.2458,245.3331,737.5619,4787.9021,2144.7114,17821.8093
572,2952.1629,200.0,1902.2624,1334.8416,4635.2941,925.7919,9858.8235,1237.1041,1302.2624,7250.6787,...,593.6652,143.8914,565.6109,303.0769,7069.6742,229.8643,607.2398,4186.4253,2135.7466,21454.2986


In [30]:
df.to_csv(r'C:\Users\Macie\Praca_Dyplomowa\ImportantValues.RNA.csv',sep=';', index=False)